# Installs

In [6]:
!pip install chroma-haystack

In [1]:
!pip install transformers[torch,sentencepiece]

In [2]:
!pip install "sentence-transformers>=3.0.0"

In [3]:
!pip install haystack-ai datasets ollama-haystack gradio

In [4]:
!pip install ollama-haystack

# Imports

In [3]:
import pandas as pd

In [4]:
from datasets import load_dataset
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers import InMemoryBM25Retriever
from haystack.components.builders import PromptBuilder
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.document_stores.types import DuplicatePolicy
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.rankers import TransformersSimilarityRanker
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.components.joiners.document_joiner import DocumentJoiner

/home/luis/UFV/RAG - LLM - Jupyter/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/luis/UFV/RAG - LLM - Jupyter/myenv/lib/python3.12/site-packages/haystack/core/errors.py:34: DeprecationWarning: PipelineMaxLoops is deprecated and will be remove in version '2.7.0'; use PipelineMaxComponentRuns instead.
  warnings.warn(


In [17]:
from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever

# Datasets

In [9]:
import pandas as pd

In [10]:
df_vuln = pd.read_parquet('/home/luis/UFV/RAG - LLM - Jupyter/dataset_reentrancy_small_multilabel.parquet')
df_vuln = df_vuln.drop_duplicates()

df_safe = pd.read_parquet('/home/luis/UFV/RAG - LLM - Jupyter/dataset_safe_small_multilabel.parquet')
df_safe = df_safe.drop_duplicates()

df_total = pd.concat([df_vuln, df_safe], ignore_index=True)
df_total = df_total.sort_values(by='source_code', key=lambda col: col.str.len(), ascending=True)
df_total.head(5)

,source_code,slither
3757,contract noFoundationProposal {},safe
1854,pragma solidity ^0.4.21;\n\n\n\ncontract Press...,safe
1630,pragma solidity ^0.4.11;\n\ncontract IconomiBl...,safe
2184,pragma solidity ^0.4.19;\n\n//Created by Moozi...,safe
3505,//SPDX-License-Identifier: Unlicense\npragma s...,safe


In [11]:
# Filtrar os contratos "safe" e "reentrancy"
safe_contracts = df_total[df_total['slither'].str.contains("safe")].head(250)
reentrancy_contracts = df_total[df_total['slither'].str.contains("reentrancy")].head(250)

# Concatenar os contratos de teste
test_df = pd.concat([safe_contracts, reentrancy_contracts])
test_df = test_df.sort_values(by='source_code', key=lambda col: col.str.len(), ascending=True)

# Selecionar contratos restantes para treino
train_df = df_total[~df_total.index.isin(test_df.index)]

# Verificar o tamanho dos DataFrames
print("Tamanho do conjunto de treino:", len(train_df))
print("Tamanho do conjunto de teste:", len(test_df))

# Garantir que a divisão está correta
assert len(train_df) + len(test_df) == len(df_total)


Tamanho do conjunto de treino: 3269
Tamanho do conjunto de teste: 500


# Pipeline

In [12]:
document_store = ChromaDocumentStore()

documents = [
    Document(
        content=row['source_code'],
        meta={
            "vulnerability": "reentrancy" if "reentrancy" in row['slither'] else "safe" if "safe" in row['slither'] else row['slither']
        }
    )
    for index, row in train_df.iterrows()
]

document_embedder  = SentenceTransformersDocumentEmbedder(model="mixedbread-ai/deepset-mxbai-embed-de-large-v1")
#document_embedder  = SentenceTransformersDocumentEmbedder()
document_embedder.warm_up()

documents_with_embeddings = document_embedder.run(documents )["documents"]
document_store.write_documents(documents_with_embeddings)

Batches: 100%|████████████████████████████████| 103/103 [02:46<00:00,  1.61s/it]


3269

In [13]:
template = """
You are an AI auditor specialized in analyzing smart contracts to detect vulnerabilities using a Retrieval-Augmented Generation (RAG) system. The following information has been retrieved from a knowledge base related to possible reentrancy issues in smart contracts.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
<|eot_id|>

Carefully inspect the provided smart contract code, focusing solely on identifying reentrancy vulnerabilities. Respond **only** with a single word: "Yes" or "No". Avoid any explanations or additional details. Provide just one word: "Yes" or "No".

Smart Contract Code: {{query}}
<|eot_id|>

"""

In [19]:
text_embedder = SentenceTransformersTextEmbedder(model="mixedbread-ai/deepset-mxbai-embed-de-large-v1")

embedding_retriever = ChromaEmbeddingRetriever(document_store=document_store, top_k = 1)

#bm25_retriever = InMemoryBM25Retriever(document_store=document_store, top_k = 1)

#document_joiner = DocumentJoiner(join_mode="merge")

ranker = TransformersSimilarityRanker(top_k = 1)

prompt_builder = PromptBuilder(template=template)

llm = OllamaGenerator(model="codellama:7b",
                            url = "http://localhost:11434",
                            generation_kwargs={
                              "num_predict": 100,
                              "temperature": 0.1,
                              "timeout": 500
                              #"max_tokens": 200,
                              #"stop": ["<|eot_id|>"]
                              })

In [20]:
rag_pipeline = Pipeline()

rag_pipeline.add_component("text_embedder", instance = text_embedder)
rag_pipeline.add_component("embedding_retriever", embedding_retriever)
#rag_pipeline.add_component("bm25_retriever", bm25_retriever)
#rag_pipeline.add_component("document_joiner", document_joiner)
rag_pipeline.add_component("ranker", ranker)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("generator", instance = llm)

In [21]:
rag_pipeline.connect("text_embedder", "embedding_retriever")
#rag_pipeline.connect("bm25_retriever", "document_joiner")
#rag_pipeline.connect("embedding_retriever", "document_joiner")
#rag_pipeline.connect("document_joiner", "ranker")
rag_pipeline.connect("embedding_retriever", "ranker")
rag_pipeline.connect("ranker", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - embedding_retriever: ChromaEmbeddingRetriever
  - ranker: TransformersSimilarityRanker
  - prompt_builder: PromptBuilder
  - generator: OllamaGenerator
🛤️ Connections
  - text_embedder.embedding -> embedding_retriever.query_embedding (List[float])
  - embedding_retriever.documents -> ranker.documents (List[Document])
  - ranker.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)

# Execução

In [22]:
question="""
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract SimpleBank {
    mapping(address => uint256) public balances;

    function deposit() public payable {
        balances[msg.sender] += msg.value;
    }

    function withdraw(uint256 amount) public {
        require(balances[msg.sender] >= amount, "Insufficient balance");

        (bool success, ) = msg.sender.call{value: amount}("");
        require(success, "Transfer failed");

        balances[msg.sender] -= amount;
    }

    receive() external payable {}
}
"""

In [24]:
resposta = rag_pipeline.run(
  {
      #"document_embedder": {"documents": documents_with_embeddings},
      #"document_joiner": {"documents": documents_with_embeddings},
      "text_embedder": {"text": question},
      "ranker": {"query": question},
      #"bm25_retriever": {"query": question},
      "prompt_builder": {"query": question}
  }
)
print(resposta["generator"]["replies"][0])

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 29.68it/s]


  Yes


In [26]:
TP, FP, TN, FN = 0, 0, 0, 0

# Itera sobre o DataFrame
for index, row in test_df.iterrows():
    resposta = rag_pipeline.run(
        {
            #"document_joiner": {"documents": documents_with_embeddings},
            "text_embedder": {"text": row["source_code"]},  # Usando a coluna "source_code"
            "ranker": {"query": row["source_code"]},
            #"bm25_retriever": {"query": row["source_code"]},
            "prompt_builder": {"query": row["source_code"]}
        }
    )
    
    llm_resposta = resposta['generator']['replies'][0].strip().lower()  # Resposta da LLM em minúsculas
    print(f"LLM linha {index}: {llm_resposta}")
    
    # Verifica se a coluna "slither" tem a palavra "reentrancy"
    if "reentrancy" in row["slither"].lower():
        slither_resposta = 'yes'
    else:
        slither_resposta = 'no'
    print(f"Slither linha {index}: {slither_resposta}")
    
    # Verifica se a resposta da LLM contém "yes" ou "no"
    if "yes" in llm_resposta:
        llm_resposta_final = 'yes'
    elif "no" in llm_resposta:
        llm_resposta_final = 'no'
    else:
        llm_resposta_final = None  # Para lidar com respostas inesperadas

    # Calcula as métricas de TP, FP, TN e FN
    if slither_resposta == 'yes' and llm_resposta_final == 'yes':
        TP += 1
    elif slither_resposta == 'no' and llm_resposta_final == 'yes':
        FP += 1
    elif slither_resposta == 'no' and llm_resposta_final == 'no':
        TN += 1
    elif slither_resposta == 'yes' and llm_resposta_final == 'no':
        FN += 1
    
    # Exibe as métricas após cada iteração
    print(f"Metrics after line {index}: TP={TP}, FP={FP}, TN={TN}, FN={FN}\n")


Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 40.36it/s]


LLM linha 3757: no
Slither linha 3757: no
Metrics after line 3757: TP=0, FP=0, TN=1, FN=0



Batches: 100%|███████████████████████████████████| 1/1 [00:00<00:00, 106.71it/s]


LLM linha 1854: yes
Slither linha 1854: no
Metrics after line 1854: TP=0, FP=1, TN=1, FN=0



Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 81.04it/s]


...

LLM linha 860: this is a solidity smart contract that manages the game of "sacrifice" on the ethereum blockchain. the contract has several functions, including:

* `buy`: allows players to buy into the game by sending ether to the contract.
* `withdraw`: allows players to withdraw their dividends and any accumulated ether they have in the contract.
* `mydividends`: returns the amount of dividends that
Slither linha 860: yes
Metrics after line 860: TP=16, FP=187, TN=60, FN=57



Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 17.49it/s]


LLM linha 1234: this is a solidity smart contract for a lottery game. it has several functions that allow users to participate in the lottery, view their bets, and view the winning numbers. the contract also has a function to generate the winning numbers randomly.

here are some key points about this code:

* the `guestbet` struct is used to store information about each guest's bet. it includes the round number, the guessed numbers, the amount
Slither linha 1234: yes
Metrics after line 1234: TP=16, FP=187, TN=60, FN=57



Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 16.17it/s]


LLM linha 516: this is a contract that allows users to create and execute transactions on behalf of another user, known as the "owner". the contract uses the oasis direct proxy pattern to implement this functionality.

here's a breakdown of the code:

1. `using dsauth` - this line imports the `dsauth` library, which provides the `auth()` function that is used to authenticate the owner of the proxy.
2. `using ds
Slither linha 516: yes
Metrics after line 516: TP=16, FP=187, TN=60, FN=58



In [27]:
accuracy = (TP + TN) / (TP + FP + TN + FN)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

print("accuracy", accuracy)
print("precision", precision)
print("recall", recall)
print("f1", f1)

accuracy 0.2367601246105919
precision 0.07881773399014778
recall 0.21621621621621623
f1 0.11552346570397111
